In [1]:
import gc
import os
import json
import torch
import pickle
import pandas as pd
import transformers
from tqdm import tqdm
from openai import AzureOpenAI
from hallucination_editor import *

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
def load_api_key(key, file_path='api_key.json'):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data[key]

def get_response(model, tok, messages, max_new_tokens=1):
    terminators = [tok.eos_token_id, tok.convert_tokens_to_ids("<|eot_id|>")]
    msg_tokenized = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt').to(model.device)
    output_ids = model.generate(msg_tokenized, max_new_tokens=max_new_tokens, eos_token_id=terminators, do_sample=False, pad_token_id=tok.eos_token_id)
    return tok.decode(output_ids[0][msg_tokenized.shape[-1]:], skip_special_tokens=True).replace('\n', ' ').strip().rstrip('.')  # remove trailing period

device = 'cuda:2'
topic_ls = ['places_city', 'places_country', 'places_landmark', 'entertainment_anime', 'entertainment_song', 'entertainment_music_genre', 'human_actor',
            'art_literary', 'art_sculpture', 'health_treatment', 'health_medication', 'health_disease', 'human_politician', 'human_writer', 'human_scientist', 
            'event_sport', 'event_history', 'event_film']
model_ls = ['meta-llama/Meta-Llama-3.1-8B-Instruct', 'mistralai/Mistral-7B-Instruct-v0.3', 'lmsys/vicuna-7b-v1.5']
model_id_format = model_ls[0].split('/')[-1].replace('-', '_').lower()
print(f'model_id: {model_id_format}')

folder_unfiltered = f"../data/questions/unfiltered/{model_id_format}"
folder_hallu = f"../data/questions/hallucination_all/{model_id_format}"

system_msg_qa, system_msg_eval

model_id: meta_llama_3.1_8b_instruct


('Always respond to the input question concisely with a short phrase or a single-word answer. Do not repeat the question or provide any explanation.',
 "Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically; otherwise, output '0'. Do not repeat the question or provide any explanation.")

In [4]:
remove_relation = ["topic's main category", "topic's main template", "described by source", "Commons category", "on focus list of Wikimedia project"]

for filename in os.listdir(folder_unfiltered):
    df = pd.read_csv(f"{folder_unfiltered}/{filename}")
    df_dup = df[df.duplicated(['subject', 'relation'], keep=False)]
    if len(df_dup) > 0:  # check duplicate (subject, relation) pairs
        print(f"In {filename}, there are {len(df_dup)} questions with duplicate (subject, relation) pairs:")
        
    if len(df[df['subject'] == df['object']]) > 0:  # Check if subject == object
        print(f"In {filename}, there are {len(df[df['subject'] == df['object']])} questions where subject == object")
        df = df[df['subject'] != df['object']]

    for relation_check in remove_relation:
        if relation_check in df['subject'].to_list():
            print(f'Check {relation_check} relation for {filename}')
            
    if len(df[df['label'] != df['object']]) > 0:
        print(f"In {filename}, there are {len(df[df['label'] != df['object']])} questions where label != object")
    df.to_csv(f"{folder_unfiltered}/{filename}", index=False)

In [ ]:
for filename in os.listdir(folder_hallu):
    df_hallu = pd.read_csv(f"{folder_hallu}/{filename}")
    if len(df_hallu[df_hallu['label'] != df_hallu['object']]) > 0:
        print(f"In {filename}, there are {len(df_hallu[df_hallu['label'] != df_hallu['object']])} questions where label != object")
    df_hallu = df_hallu.drop(columns=['label'])  # label column equals to the object column
    df_hallu.to_csv(f"{folder_hallu}/{filename}", index=False)

## Get responses

In [4]:
tok_qa = transformers.AutoTokenizer.from_pretrained(model_id)
model_qa = transformers.AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
messages_qa = [{"role": "system", "content": system_msg_qa}, {"role": "user", "content": 'What does Vulkan follow?'}]
get_response(model_qa, tok_qa, messages_qa, max_new_tokens=16)

In [5]:
print(model_id_format)
# df_all_topics = pd.DataFrame()
# for filename in ['technology_software.csv']:
for filename in os.listdir(folder_unfiltered):
    df = pd.read_csv(f"{folder_unfiltered}/{filename}")
    # if f"output_{model_id_format}" in df.columns:
    #     continue
    ls_output = []
    for i in tqdm(df.index):
        question = df.loc[i, 'question']
        # user_msg_qa = Wh_content + "\nQuestion:" + question  # places_landmark_old.csv
        # user_msg_qa = f'Question: {question} Answer:'
        user_msg_qa = f'{question}'
        if model_id_format == 'gemma_2_9b_it':  # System role not supported for gemma
            messages_qa = [{"role": "user", "content": system_msg_qa+' '+user_msg_qa}]
        else:
            messages_qa = [{"role": "system", "content": system_msg_qa}, {"role": "user", "content": user_msg_qa}]
        output_qa = get_response(model_qa, tok_qa, messages_qa, max_new_tokens=16)
        ls_output.append(output_qa)
    
    df['topic'] = filename.replace('.csv', '')
    df[f"output_{model_id_format}"] = ls_output
    df[['topic', 'subject', 'relation', 'object', 'label', 'question', f'output_{model_id_format}']].to_csv(f"{folder_unfiltered}/{filename}", index=False)
    print(filename)
    # df_all_topics = pd.concat([df_all_topics, df], axis=0)
    # print("df_all_topics.shape:", df_all_topics.shape)
# df_all_topics = df_all_topics[['topic', 'type', 'subject', 'relation', 'object', 'question', 'label', f'output_{model_id_format}']]
# df_all_topics.to_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv", index=False)
# del model_qa
# gc.collect()
# torch.cuda.empty_cache() 190 min

meta_llama_3.1_8b_instruct


  0%|          | 0/115 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 115/115 [00:50<00:00,  2.30it/s]


event_sport.csv


100%|██████████| 261/261 [01:54<00:00,  2.27it/s]


...

places_country.csv


100%|██████████| 1546/1546 [10:22<00:00,  2.48it/s]


human_entrepreneur.csv


100%|██████████| 107/107 [00:54<00:00,  1.98it/s]

geography_glacier.csv


## Filter hallucination

In [6]:
def evaluate_responses(model_eval, tok_eval, df, system_msg_eval, user_msg_eval_template="Text 1: {label} \nText 2: {output_qa}"):
    for i in df.index:
        label = df.loc[i, 'object']
        output_qa = df.loc[i, f"output_{model_id_format}"]
        eval_res = 0

        if output_qa.lower() in label.lower() or label.lower() in output_qa.lower() or 'unknown' in output_qa.lower():  # Rule-based fuzzy match
            eval_res = 1
            # if output_qa.lower() == label.lower():
            #     print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Exact Match")
            # else:
            #     print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Partial Match")
        else:
            user_msg_eval = user_msg_eval_template.format(label=label, output_qa=output_qa)
            messages_eval = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": user_msg_eval}]
            response_eval = get_response(model_eval, tok_eval, messages_eval)
            if response_eval != '0':
                # print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Semantic Match")
                eval_res = 1
                
        df.loc[i, f"eval_{model_id_format}"] = eval_res
    hallu_count = df[df[f'eval_{model_id_format}']==0].shape
    print(f"Hallucination ratio: {hallu_count[0]/len(df)} df_hallucination.shape: {hallu_count}")
    return df

print(system_msg_eval)
model_id_eval = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_eval = AutoModelForCausalLM.from_pretrained(model_id_eval, torch_dtype='auto').to('cuda:3')
tok_eval = AutoTokenizer.from_pretrained(model_id_eval)

Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically; otherwise, output '0'. Do not repeat the question or provide any explanation.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# domain_topic_name = 'entertainment_anime' 
# df_q = df_q[df_q.topic == domain_topic_name]
system_msg_2 = "Given two texts, labeled as Text 1 and Text 2, output '1' if they if they have similar semantic meanings, are synonyms, \
or if one is a more specific or general version of the other; otherwise, output '0'. Do not repeat the question or provide any explanation."   

if not os.path.exists(folder_hallu):
    os.makedirs(folder_hallu)
# for filename in ['technology_software.csv', 'places_country.csv', 'business_brand.csv', 'human_scientist.csv']:
for filename in os.listdir(folder_unfiltered)[:]:
    print(filename, os.path.exists(f"{folder_hallu}/{filename}"))
    if os.path.exists(f"{folder_hallu}/{filename}"):
        continue
    df_q = pd.read_csv(f"{folder_unfiltered}/{filename}")
    print(f"domain_topic_name: {filename}, df_q.shape: {df_q.shape}, output_dir: {folder_hallu}/{filename}")
    print('Round 1.', end=' ')
    df_q = evaluate_responses(model_eval, tok_eval, df_q, system_msg_eval)
    df_hallu = df_q[df_q[f"eval_{model_id_format}"] == 0]

    # Round 2: use system_msg_2 to filter case such as United Kingdom vs. England
    print('Round 2.', end=' ')
    df_hallu = evaluate_responses(model_eval, tok_eval, df_hallu, system_msg_2)
    df_hallu = df_hallu[df_hallu[f"eval_{model_id_format}"] == 0]
    df_hallu = df_hallu.drop(columns=['label'])  # label column equals to the object column
    df_hallu.to_csv(f"{folder_hallu}/{filename}", index=False)

### Other evaluation less/more strict

In [ ]:
# system_msg_eval = """Given a question, a label, and a prediction, evaluate the correctness of the prediction compared to the label. \
# Output '1' if they have similar semantic meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'. \
# Only output the final evaluation as a single word. Do not repeat the question or provide an explanation."""

In [ ]:
domain_topic_name = 'places_landmark'
df_wh = pd.read_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv")
df_wh = df_wh[df_wh.topic == domain_topic_name]
print(domain_topic_name, df_wh.shape)

system_msg_eval = """Given a label and a prediction, evaluate the correctness of the prediction compared to the label. \
Output '1' if they match each other semantically. Otherwise, output '0'. Do not repeat the question or provide an explanation."""
# output the final evaluation as a single word. Do not repeat the question or provide an explanation Only output '0' or '1'.

user_msg_eval_template = f"""label: {label} \nprediction: {output_qa}\n"""
df_wh = evaluate_responses(model_eval, tok_eval, df_wh, system_msg_eval, user_msg_eval_template)

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 
The wh question accuracy of the language model is 0.418


In [ ]:
domain_topic_name = 'places_landmark'
df_wh = pd.read_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv")
df_wh = df_wh[df_wh.topic == domain_topic_name]

system_msg_eval = """Given a question, a label, and a prediction, evaluate the correctness of the prediction compared to the label. \
Output '1' if they have similar semantic meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'. \
Only output the final evaluation as a single word. Do not repeat the question or provide an explanation."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label, output_qa = df_wh.loc[i, 'question'], df_wh.loc[i, 'label'], df_wh.loc[i, f"output_{model_id_format}"]
    prompt_eval = f"""question: {question} \nlabel: {label} \nprediction: {output_qa}\n"""
    eval_res = 0
    wh_count += 1 

    if output_qa.lower() in label.lower() or label.lower() in output_qa.lower():  # Rule-basd fuzzy match
        wh_correct += 1
        eval_res = 1
    else:
        
        messages_eval = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": user_msg_eval}]
        response_eval = get_response(model_eval, tok_eval, messages_eval)
        if response_eval == '1':
            wh_correct += 1
            eval_res = 1
            
    df_wh.loc[i, f"eval_{model_id_format}"] = eval_res
    
print(f"The wh question accuracy of the language model is {wh_correct / wh_count}")
# if not os.path.exists(f"../data/questions/wh_only/hallucination_only/{model_id_format}"):
#     os.makedirs(f"../data/questions/wh_only/hallucination_only/{model_id_format}")
df_wh[df_wh[f"eval_{model_id_format}"] == 0].to_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}.csv", index=False)
df_wh[df_wh[f"eval_{model_id_format}"] == 1]
# The wh question accuracy of the language model is 0.656

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The wh question accuracy of the language model is 0.642


,topic,type,subject,relation,object,question,label,output_meta_llama_3.1_8b_instruct,eval_meta_llama_3.1_8b_instruct
2000,places_landmark,wh,Old Royal Naval College,architect,Christopher Wren,Who does Old Royal Naval College architect?,Christopher Wren,Christopher Wren.,1.0
2001,places_landmark,wh,Old Royal Naval College,country,United Kingdom,What is the country of Old Royal Naval College?,United Kingdom,United Kingdom.,1.0
2003,places_landmark,wh,Panathenaic Stadium,occupant,Hellenic Olympic Committee,What is the occupant of Panathenaic Stadium?,Hellenic Olympic Committee,Greek National Stadium.,1.0
2004,places_landmark,wh,Panathenaic Stadium,made from material,marble,What is the made from material of Panathenaic ...,marble,Pentelic marble.,1.0
2005,places_landmark,wh,Panathenaic Stadium,sponsor,Stavros Niarchos Foundation,What does Panathenaic Stadium sponsor?,Stavros Niarchos Foundation,Athletic events.,1.0
...,...,...,...,...,...,...,...,...,...
2492,places_landmark,wh,MUNCH,located in or next to body of water,Oslofjord,What is the located in or next to body of wate...,Oslofjord,Oslofjord.,1.0
2494,places_landmark,wh,MUNCH,replaces,Munch Museum,What does MUNCH replace?,Munch Museum,MUNCH replaces OMNISCIENT.,1.0
2496,places_landmark,wh,Bridget of Sweden,founded by,Vadstena Abbey,Which tourist attraction was founded by Bridge...,Vadstena Abbey,Birgitta Abbey.,1.0
2498,places_landmark,wh,St Mark's Campanile,has part(s),St Mark's Basilica,Which tourist attraction has part(s) St Mark's...,St Mark's Basilica,St. Mark's Square.,1.0


In [7]:
# May output other than 0 or 1
# system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate if the prediction and the correct answer match semantically. \
# Output '1' if they have similar meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'."""

system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate whether the prediction and the correct answer match semantically. \
Output '1' if they convey similar meanings, including when the prediction is more specific, more general, or a synonym of the correct answer. Otherwise, output '0'."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label, output = df_wh.loc[i, 'question'], df_wh.loc[i, 'label'], df_wh.loc[i, f"output_{model_id}"]
    prompt_eval = f"""The inputs are given as below: \nquestion: {question} \n\ncorrect answer: {label} \n\nprediction: {output}\n"""

    eval_res = 0
    wh_count += 1

    if output.lower() in label.lower() or label.lower() in output.lower():  # Rule-basd fuzzy match
        wh_correct += 1
        eval_res = 1
    else:
        messages = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval+" Only output '1' or '0'."}]
        msg_tokenized = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt')
        output_ids = model_eval.generate(msg_tokenized.to(device_eval), max_new_tokens=1, eos_token_id=terminators, do_sample=False, pad_token_id=tok.eos_token_id)
        response_str = tok.decode(output_ids[0][msg_tokenized.shape[-1]:], skip_special_tokens=True)

        if response_str == '1':
            wh_correct += 1
            eval_res = 1
            
    df_wh.loc[i, f"eval_{model_id_format}"] = eval_res
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh[df_wh[f"eval_{model_id_format}"] == 0].to_csv(f"../data/questions/wh_only/hallucination_only/{domain_topic_name}_{model_id_format}_eval.csv", index=False)
df_wh[df_wh[f"eval_{model_id_format}"] == 1]

/home/baix/anaconda3/envs/env24may/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/baix/anaconda3/envs/env24may/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[GPT] The wh question accuracy of the language model is 0.732


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta_llama_3_8b_instruct
0,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
2,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
3,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
4,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,1.0
497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,1.0


In [16]:
from sentence_transformers import SentenceTransformer, util

ls_label = df_wh.label.tolist()

wh_count = 0
wh_correct = 0
model_name = 'paraphrase-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    flag = False

    wh_count += 1
    embeddings = model.encode([label, output])
    similarity_score = util.cos_sim(embeddings[0], embeddings[1])
    threshold = 0.6
    if similarity_score >= threshold:
        wh_correct += 1
        flag = True
        
wh_acc_dict = {"wh_accuracy": wh_correct/wh_count}
print(f"[Sentence Transformer] The wh question accuracy of the language model is {wh_correct / wh_count}")

[Sentence Transformer] The wh question accuracy of the language model is 0.418


In [18]:
llm_name = 'gpt-35-turbo'

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
    info = "Question: "+question+ "\nModel Answer: "+label + "\nAI model generated answer: " + output \
        + "\n. The above question's topic is " + domain_topic_name  + ". "
    string = """\n Please evaluate the correctness of the AI model's answer compared to the model answer. 
        Consider the following criteria and provide your judgment:
        If the AI's answer is a more specific version of the model answer, please respond with: "Correct"
        If the AI's answer is a more general version of the model answer, please respond with: "Correct".
        If the AI's answer is a closely related to the model answer, please respond with: "Correct".
        If the AI's answer and the model answer are entirely different entities with no direct relationship, please respond with: "Incorrect".
        """

    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": ""}, {"role": "user", "content": info + string}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')
    # response_str

    if response_str and response_str.rstrip('.') == "Correct":
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[GPT] The wh question accuracy of the language model is 0.694


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,1.0


In [19]:
llm_name = 'gpt-35-turbo'
topic = 'health_medication'

system_msg_eval = "Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically, and output '0' if they do not."

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
    # info = "Question: "+question+ "\nModel Answer: "+label + "\nAI model generated answer: " + output \
    #     + "\n. The above question's topic is " + topic  + ". "
    prompt_eval = f"""The input texts are given as below: \nText 1: {label} \n\nText 2: {output}\n"""
    
    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')

    if str(response_str) == '1':
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

[GPT] The wh question accuracy of the language model is 0.252


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,0.0


In [21]:
df_wh[df_wh[f"eval_{model_id}"]==1]

,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
1012,wh,John Rylands Library,country,United Kingdom,What is the country of John Rylands Library?,United Kingdom,United Kingdom,1.0
...,...,...,...,...,...,...,...,...
1474,wh,Jōshin'etsu-kōgen National Park,located in protected area,Shiga Highlands,Which tourist attraction's located in protecte...,Shiga Highlands,Shiga Kogen,1.0
1475,wh,Mount Kilimanjaro,country,Tanzania,What is the country of Mount Kilimanjaro?,Tanzania,Tanzania,1.0
1484,wh,Night Safari,country,Singapore,What is the country of Night Safari?,Singapore,Singapore,1.0
1489,wh,St Paul's Cathedral,architect,Christopher Wren,Who does St Paul's Cathedral architect?,Christopher Wren,Christopher Wren,1.0


In [20]:
system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate whether the prediction is semantically equivalent to the correct answer. \
Output '1' if they are semantically equivalent, otherwise output '0'."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
        # + "\n. The above question's topic is " + topic  + ". "
    prompt_eval = f"""The inputs are given as below: \nquestion: {question} \n\ncorrect answer: {label} \n\nprediction: {output}\n"""
    
    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')

    if str(response_str) == '1':
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

[GPT] The wh question accuracy of the language model is 0.278


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,0.0


### Topic-specific cleaning

In [3]:
folder_hallu_100 = f"../data/questions/hallucination/{model_id_format}_100"

# for filename in ['business_brand.csv']:
for filename in sorted(os.listdir(folder_hallu)):
    df_q = pd.read_csv(f"{folder_hallu}/{filename}")
    print(f'{filename} Shape before sampling: {df_q.shape}', end=" ")
    # if not os.path.exists(folder_hallu_100):
    #     os.makedirs(folder_hallu_100)
    if os.path.exists(f"{folder_hallu_100}/{filename}"):
        print(f"{filename} already exists")
        continue
        # df_q100 = pd.read_csv(f"{folder_hallu_100}/{filename}")
        # if 'paraphrased_question' in df_q100.columns:
        #     continue
    if len(df_q) > 100:
        df_q.sample(100, random_state=28).to_csv(f"{folder_hallu_100}/{filename}", index=False)
        # print(100)
    else:
        df_q.to_csv(f"{folder_hallu_100}/{filename}", index=False)
    print(f"Saving to {folder_hallu_100}/{filename}")

art_sculpture.csv Shape before sampling: (243, 7) art_sculpture.csv already exists
business_brand.csv Shape before sampling: (371, 7) business_brand.csv already exists
business_corporation.csv Shape before sampling: (106, 7) business_corporation.csv already exists
business_industry.csv Shape before sampling: (138, 7) business_industry.csv already exists
entertainment_anime.csv Shape before sampling: (184, 7) entertainment_anime.csv already exists
entertainment_music_genre.csv Shape before sampling: (158, 7) entertainment_music_genre.csv already exists
entertainment_song.csv Shape before sampling: (1579, 7) entertainment_song.csv already exists
event_film.csv Shape before sampling: (49, 7) event_film.csv already exists
event_history.csv Shape before sampling: (93, 7) event_history.csv already exists
event_sport.csv Shape before sampling: (35, 7) event_sport.csv already exists
geography_forest.csv Shape before sampling: (168, 7) geography_forest.csv already exists
geography_glacier.csv S

In [174]:
def clean_relation(domain_topic_name, relation_ls):
    df = pd.read_csv(f"{folder_hallu}/{domain_topic_name}.csv")
    df_q100 = pd.read_csv(f"{folder_hallu_100}/{domain_topic_name}.csv")
    if 'paraphrased_question' in df_q100.columns:
        print(f'Full set of questions exists for {domain_topic_name}')
        return
    df_new = df[~df['relation'].isin(relation_ls)]
    print(f"{len(df) - len(df_new)} rows removed", end=" ")
    # df_new.to_csv(f"{folder_hallu}/{domain_topic_name}.csv", index=False)

    if len(df_new) > 100:
        df_new = df_new.sample(100, random_state=28)
        df_new.to_csv(f"{folder_hallu_100}/{domain_topic_name}.csv", index=False)
        print(f"Saving to {folder_hallu_100}/{domain_topic_name}.csv")

clean_relation('event_sport', relation_ls=['sponsor', 'has part(s)'])
clean_relation('event_film', relation_ls=['sponsor', 'has part(s)'])
clean_relation('event_history', relation_ls=['sponsor', 'has part(s)'])

relation_ls = ['described by source', 'student', 'child', 'doctoral student', 'educated at', 'given name'] 
clean_relation('human_writer', relation_ls=relation_ls)
clean_relation('human_athlete', relation_ls=relation_ls)
clean_relation('human_scientist', relation_ls=relation_ls)
clean_relation('human_entrepreneur', relation_ls=relation_ls)

relation_ls = ['twinned administrative body', 'flag', 'history of topic', 'executive body', 'studied in', 'public holiday', 'economy of topic', 
               'geography of topic', 'described by source', 'demographics of topic', 'has part(s)', 'follows', 'diplomatic relation', 'culture',
               'present in work']
clean_relation('places_city', relation_ls=relation_ls)
clean_relation('places_country', relation_ls=relation_ls)
clean_relation('places_landmark', relation_ls=relation_ls)

clean_relation('technology_software', relation_ls=['platform'])
clean_relation('business_brand', relation_ls=['CPU', 'participant'])


1274 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/places_city.csv
202 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/places_country.csv
81 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/places_landmark.csv
186 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/human_writer.csv
78 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/human_athlete.csv
115 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/human_scientist.csv
111 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/human_entrepreneur.csv
34 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/technology_software.csv
3 rows removed Saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/business_brand.csv
3 rows removed 2 rows removed 5 rows

### Generate other types of questions

In [194]:
domain_topic_name = ['places_country', 'business_brand', 'human_scientist'][2]
client = AzureOpenAI(api_key=load_api_key('api_key_n_central_us'), api_version='2023-05-15', azure_endpoint="https://n-central-us.openai.azure.com/")
df_hallu = pd.read_csv(f"{folder_hallu_100}/{domain_topic_name}.csv")
df_hallu = df_hallu[df_hallu.topic==domain_topic_name]
print(len(df_hallu), domain_topic_name)

def get_gpt_response(client, system_msg, prompt, model='gpt-4o'):
    raw_response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": prompt}], 
        response_format={"type": "json_object"},
        temperature=0
    )
    return raw_response

df_hallu.columns

100 human_scientist


Index(['topic', 'subject', 'relation', 'object', 'label', 'question',
       'output_mistral_7b_instruct_v0.3', 'eval_mistral_7b_instruct_v0.3'],
      dtype='object')

In [195]:
system_msg_gen_q = """Given a fact triplet (subject, relation, object), a question asking for the object, and a wrong answer, the correct answer to the question should be the object in the triplet. Generate the following types of questions:
1. Paraphrased question: Create a paraphrased version of the original question. The correct answer should still be the object from the triplet.
2. Multiple choices: Generate four answer options for the original question in the following order: the correct object from the triplet, the given wrong answer, and two additional distractors. 
3. Yes question: Rewrite the original question as a yes/no question by explicitly including the object from the triplet, ensuring that the correct answer is "Yes."
4. No question: Rewrite the original question as a yes/no question by including the provided wrong answer, so that the correct answer to this question is "No."
5. Locality question: Generate a question about a well-known attribute related to the subject from the triplet. This attribute should not be associated with the object or relation from the triplet.
6. Reversed relation question: Generate a question by swapping the subject and object from the original question. The answer should now be the subject from the triplet.
Output the result in JSON format with the following keys: "paraphrased_question", "multiple_choices", "yes_question", "no_question", "locality_question", and "reversed_relation_question."\
"""
# system_msg_gen_q = """Given a fact triplet (subject, relation, object), a question asking for the object, and a wrong answer, the correct answer to the question should be the object in the triplet. Generate the following types of questions:
# 1. Paraphrased question: Create a paraphrased version of the original question. The correct answer should still be the object from the triplet.
# 2. Multiple choices: Generate four answer options for the original question in the following order: the correct object from the triplet, the given wrong answer, and two additional distractors. 
# Output the result in JSON format with the following keys: "paraphrased_question", "multiple_choices"\
# """
# Portability question: Create a portability question by replacing the subject in the original question with an alias or synonym. 
# 4. Multiple-Choice Question: Create a question with four answer options. Include the correct answer (the object in the triplet), \
# one provided wrong answer, and two additional distractors. Randomly assign these options to choices A, B, C, and D. Use the following JSON format: \
# {"question": "...", "options": {"A": "...", "B": "...", "C": "...", "D": "..."}, "ground_truth": "Only provide the correct answer as a letter (A, B, C, or D)"}

paraphrased_questions, multiple_choices, yes_questions, no_questions, locality_questions, reversed_relation_questions = ([] for _ in range(6))


def expand_questions(df_hallu):
    df_t = None
    df_t_list = []
    
    other_model_path = f"../data/questions/hallucination/meta_llama_3.1_8b_instruct_100/{domain_topic_name}.csv"
    if os.path.exists(other_model_path):
        df_t_list.append(pd.read_csv(other_model_path))
    if os.path.exists(f"{folder_hallu_100}/{domain_topic_name}_check.csv"):
        df_t_list.append(pd.read_csv(f"{folder_hallu_100}/{domain_topic_name}_check.csv"))
    df_t = pd.concat(df_t_list, ignore_index=True)
    # hallucination from other model that may contain questions for same fact triplets
    print(f'Data may contain already generated questions df_t.shape: {df_t.shape}')

    for i in df_hallu.index[:]:
        subject, relation, object, question = df_hallu.loc[i, 'subject'], df_hallu.loc[i, 'relation'], df_hallu.loc[i, 'object'], df_hallu.loc[i, 'question']
        if df_t is not None:
            matching_row = df_t[(df_t['topic']==domain_topic_name) & (df_t['subject']==subject) & (df_t['relation']==relation) & (df_t['object']==object)]
            if not matching_row.empty:
                print(matching_row.to_dict())
                paraphrased_questions.append(matching_row['paraphrased_question'].values[0])
                multiple_choices.append(eval(matching_row['multiple_choices'].values[0]))
                yes_questions.append(matching_row['yes_question'].values[0])
                no_questions.append(matching_row['no_question'].values[0])
                locality_questions.append(matching_row['locality_question'].values[0])
                reversed_relation_questions.append(matching_row['reversed_relation_question'].values[0])
                continue

        pre_edit_ans = df_hallu.loc[i, f'output_{model_id_format}']
        prompt_gen_q = f"subject: {subject}, relation: {relation}, object: {object}, question: {question}, wrong answer: {pre_edit_ans}"
        raw_response = get_gpt_response(client, system_msg_gen_q, prompt_gen_q)
        json_obj = json.loads(raw_response.choices[0].message.content)
        print(json_obj)
        paraphrased_questions.append(json_obj['paraphrased_question'])
        multiple_choices.append(json_obj['multiple_choices'])
        yes_questions.append(json_obj['yes_question'])
        no_questions.append(json_obj['no_question'])
        locality_questions.append(json_obj['locality_question'])
        reversed_relation_questions.append(json_obj['reversed_relation_question'])

expand_questions(df_hallu)

Data may contain already generated questions df_t.shape: (200, 28)


09/13/2024 23:59:05 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


{'paraphrased_question': 'What gender is Berthold Vogl?', 'multiple_choices': ['male', 'Not specified in the provided information', 'female', 'non-binary'], 'yes_question': 'Is the sex or gender of Berthold Vogl male?', 'no_question': 'Is the sex or gender of Berthold Vogl not specified in the provided information?', 'locality_question': 'What is a notable achievement of Berthold Vogl?', 'reversed_relation_question': 'Who is male in this context?'}


09/13/2024 23:59:07 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


{'paraphrased_question': 'What gender is Benjamin Constant?', 'multiple_choices': ['male', 'Not applicable (Benjamin Constant was a historical figure, not a living person)', 'female', 'non-binary'], 'yes_question': 'Is the sex or gender of Benjamin Constant male?', 'no_question': 'Is the sex or gender of Benjamin Constant not applicable (Benjamin Constant was a historical figure, not a living person)?', 'locality_question': 'What is Benjamin Constant known for?', 'reversed_relation_question': 'Who is male in relation to sex or gender?'}


...

{'paraphrased_question': 'Who supervised Martin Karplus during his doctoral studies?', 'multiple_choices': ['Linus Pauling', 'John Pople', 'Richard Feynman', 'Marie Curie'], 'yes_question': 'Is Linus Pauling the doctoral advisor of Martin Karplus?', 'no_question': 'Is John Pople the doctoral advisor of Martin Karplus?', 'locality_question': 'What notable award did Martin Karplus win in 2013?', 'reversed_relation_question': 'Who was advised by Linus Pauling during their doctoral studies?'}


09/14/2024 00:01:00 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


{'paraphrased_question': 'Who served as the doctoral advisor to Isaac Beeckman?', 'multiple_choices': ['Rudolph Snellius', 'Tycho Brahe', 'Johannes Kepler', 'Galileo Galilei'], 'yes_question': 'Is Rudolph Snellius the doctoral advisor of Isaac Beeckman?', 'no_question': 'Is Tycho Brahe the doctoral advisor of Isaac Beeckman?', 'locality_question': 'What field is Isaac Beeckman known for contributing to?', 'reversed_relation_question': 'Who was advised by Rudolph Snellius?'}
{'topic': {62: 'human_scientist', 117: 'human_scientist'}, 'subject': {62: 'Buckminster Fuller', 117: 'Buckminster Fuller'}, 'relation': {62: 'movement', 117: 'movement'}, 'object': {62: 'high-tech architecture', 117: 'high-tech architecture'}, 'question': {62: 'What is the movement of Buckminster Fuller?', 117: 'What is the movement of Buckminster Fuller?'}, 'output_meta_llama_3.1_8b_instruct': {62: 'Dymaxionism', 117: nan}, 'eval_meta_llama_3.1_8b_instruct': {62: 0.0, 117: nan}, 'multiple_choices': {62: "['high-te

09/14/2024 00:01:02 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


{'paraphrased_question': 'What gender does Benedikt Buchner identify as?', 'multiple_choices': ['male', 'Not specified', 'female', 'non-binary'], 'yes_question': 'Is the sex or gender of Benedikt Buchner male?', 'no_question': 'Is the sex or gender of Benedikt Buchner not specified?', 'locality_question': 'What is a notable profession or role associated with Benedikt Buchner?', 'reversed_relation_question': 'Who is identified as male in this context?'}
{'topic': {38: 'human_scientist'}, 'subject': {38: 'Semyon Lavochkin'}, 'relation': {38: 'employer'}, 'object': {38: 'NPO Lavochkin'}, 'question': {38: 'What is the employer of Semyon Lavochkin?'}, 'output_meta_llama_3.1_8b_instruct': {38: 'OKB-301'}, 'eval_meta_llama_3.1_8b_instruct': {38: 0.0}, 'multiple_choices': {38: "['NPO Lavochkin', 'OKB-301', 'NASA', 'Roscosmos']"}, 'paraphrased_question': {38: 'Who employed Semyon Lavochkin?'}, 'multiple_choice_with_letters': {38: 'A. NPO Lavochkin  B. OKB-301  C. Roscosmos  D. NASA'}, 'multiple

In [196]:
def naive_match(label, response):
    return label.lower() in response.lower() or response.lower() in label.lower()
df_hallu['multiple_choices'] = multiple_choices
ls_multiple_choice_with_letters, ls_multiple_choice_labels = [], []
for i in df_hallu.index[:]:
    subject, relation, label, question = df_hallu.loc[i, 'subject'], df_hallu.loc[i, 'relation'], df_hallu.loc[i, 'object'], df_hallu.loc[i, 'question']
    wrong_ans = df_hallu.loc[i, f'output_{model_id_format}']
    four_choices = eval(df_hallu.loc[i, 'multiple_choices']) if type(df_hallu.loc[i, 'multiple_choices']) == str else df_hallu.loc[i, 'multiple_choices']
    choice = [label, wrong_ans, four_choices[2], four_choices[3]]
    # print(choice, type(four_choices))
    print(f"Check label: {label:<50} four_choices[0]: {four_choices[0]}") if not naive_match(label, four_choices[0]) else None
    # The use of credit-saving df_t may cause the multiple_choices from other models (built on its wrong_ans) to be different than the wrong_ans of the current model.
    print(f"Check wrong_ans: {wrong_ans:<50} four_choices[1]: {four_choices[1]}") if not naive_match(wrong_ans, four_choices[1]) else None
    MC_dict = {"0": "A", "1": "B", "2": "C", "3": "D"}
    random.shuffle(choice)
    correct_answer = MC_dict[str(choice.index(label))]
    choice_str = ""
    for i in range(4):
        choice_str += (MC_dict[str(i)] + ". " + choice[i] + "  ")
    # print(choice_str)
    ls_multiple_choice_with_letters.append(choice_str.strip())
    ls_multiple_choice_labels.append(correct_answer)

Check wrong_ans: Kader Abderhamane                                  four_choices[1]: Simon Ammann
Check wrong_ans: United States                                      four_choices[1]: Winchester
Check wrong_ans: Architecture                                       four_choices[1]: Geologist
Check wrong_ans: France                                             four_choices[1]: Paris
Check wrong_ans: San Diego, USA                                     four_choices[1]: California
Check label: Ralph H. Fowwlerr                                  four_choices[0]: Ralph H. Fowler
Check wrong_ans: Paul Ehrenfest                                     four_choices[1]: Paul Dirac's doctoral advisor was Ralph Fowler
Check wrong_ans: Herman van der Weele                               four_choices[1]: Hendrik Doeff
Check wrong_ans: Marcel Badiou                                      four_choices[1]: Louis Badiou
Check wrong_ans: Strasbourg                                         four_choices[1]: Paris
Check w

In [197]:
# df_hallu = df_hallu.iloc[:]
print(f"Before df_hallu.shape: {df_hallu.shape}")
df_hallu['paraphrased_question'] = paraphrased_questions
df_hallu['multiple_choice_with_letters'] = ls_multiple_choice_with_letters
df_hallu['multiple_choice_labels'] = ls_multiple_choice_labels
df_hallu['yes_question'] = yes_questions
df_hallu['no_question'] = no_questions
df_hallu['locality_question'] = locality_questions
df_hallu['reversed_relation_question'] = reversed_relation_questions
print(f"After df_hallu.shape: {df_hallu.shape}")
df_hallu.to_csv(f"{folder_hallu_100}/{domain_topic_name}.csv", index=False)

Before df_hallu.shape: (100, 9)
After df_hallu.shape: (100, 16)


In [198]:
client = AzureOpenAI(api_key=load_api_key('api_key_n_central_us'), api_version='2023-05-15', azure_endpoint="https://n-central-us.openai.azure.com/")
system_msg_gen_q = """ 
Given a subject and a relation, your task is to create 2-hop, 3-hop, 4-hop, 5-hop, and 6-hop questions, along with their correct answers. \
Always use the provided subject and relation to create multi-hop questions, and avoid including any correct answers from other multi-hop questions. \
Ensure that the multi-hop questions and answers are accurate, and avoid using 'N/A' or 'unknown' as responses. Output in JSON format. Below is an example:

Example input: 
subject: Amazon, relation: founder

Example output: 
{
    "2hop_question": "Who is the spouse of the Amazon founder?",
    "2hop_answer": "MacKenzie Scott",
    "3hop_question": "Which university did the spouse of the Amazon founder attend for their undergraduate studies?",
    "3hop_answer": "Princeton University",
    "4hop_question": "In which city is the university that the spouse of the Amazon founder attended located?",
    "4hop_answer": "Princeton",
    "5hop_question": "In which state is the city located where the university that the spouse of the Amazon founder attended is situated?",
    "5hop_answer": "New Jersey",
    "6hop_question": "In which country is the state located where the city is situated that contains the university the spouse of the Amazon founder attended?",
    "6hop_answer": "United States",
}
"""

ls_2hop_q, ls_2hop_a, ls_3hop_q, ls_3hop_a, ls_4hop_q, ls_4hop_a, ls_5hop_q, ls_5hop_a, ls_6hop_q, ls_6hop_a = ([] for _ in range(10))

def generate_questions(df_hallu):
    df_t = None
    df_t_list = []
    
    other_model_path = f"../data/questions/hallucination/meta_llama_3.1_8b_instruct_100/{domain_topic_name}.csv"
    if os.path.exists(other_model_path):
        df_t_list.append(pd.read_csv(other_model_path))
    if os.path.exists(f"{folder_hallu_100}/{domain_topic_name}_check.csv"):
        df_t_list.append(pd.read_csv(f"{folder_hallu_100}/{domain_topic_name}_check.csv"))
    df_t = pd.concat(df_t_list, ignore_index=True)
    print(f'Data (df_t) may contain already generated questions df_t.shape: {df_t.shape}')

    for i in df_hallu.index[:]:
        subject, relation, object, question = df_hallu.loc[i, 'subject'], df_hallu.loc[i, 'relation'], df_hallu.loc[i, 'object'], df_hallu.loc[i, 'question']
        if df_t is not None:
            matching_row = df_t[(df_t['topic']==domain_topic_name) & (df_t['subject']==subject) & (df_t['relation']==relation) & (df_t['object']==object)]
            if not matching_row.empty:
                print(matching_row.to_dict())
                ls_2hop_q.append(matching_row['question_2hop'].values[0])
                ls_2hop_a.append(matching_row['answer_2hop'].values[0])
                ls_3hop_q.append(matching_row['question_3hop'].values[0])
                ls_3hop_a.append(matching_row['answer_3hop'].values[0])
                ls_4hop_q.append(matching_row['question_4hop'].values[0])
                ls_4hop_a.append(matching_row['answer_4hop'].values[0])
                ls_5hop_q.append(matching_row['question_5hop'].values[0])
                ls_5hop_a.append(matching_row['answer_5hop'].values[0])
                ls_6hop_q.append(matching_row['question_6hop'].values[0])
                ls_6hop_a.append(matching_row['answer_6hop'].values[0])
                continue
            
        prompt_gen_q = f"subject: {subject}, relation: {relation}"
        raw_response = get_gpt_response(client, system_msg_gen_q, prompt_gen_q)
        raw_str = raw_response.choices[0].message.content
        json_obj = json.loads(raw_str)
        print(f"subject: {subject}, relation: {relation}, {json_obj}")
        ls_2hop_q.append(json_obj['2hop_question'])
        ls_2hop_a.append(json_obj['2hop_answer'])
        ls_3hop_q.append(json_obj['3hop_question'])
        ls_3hop_a.append(json_obj['3hop_answer'])
        ls_4hop_q.append(json_obj['4hop_question'])
        ls_4hop_a.append(json_obj['4hop_answer'])
        ls_5hop_q.append(json_obj['5hop_question'])
        ls_5hop_a.append(json_obj['5hop_answer'])
        ls_6hop_q.append(json_obj['6hop_question'])
        ls_6hop_a.append(json_obj['6hop_answer'])

generate_questions(df_hallu)

Data (df_t) may contain already generated questions df_t.shape: (200, 28)


09/14/2024 00:01:04 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


subject: Berthold Vogl, relation: sex or gender, {'2hop_question': 'What is the profession of Berthold Vogl?', '2hop_answer': 'Physicist', '3hop_question': 'Which university did the physicist Berthold Vogl attend?', '3hop_answer': 'University of Vienna', '4hop_question': 'In which city is the University of Vienna located?', '4hop_answer': 'Vienna', '5hop_question': 'In which country is the city of Vienna located?', '5hop_answer': 'Austria', '6hop_question': 'What is the official language of the country where the city of Vienna is located?', '6hop_answer': 'German'}


09/14/2024 00:01:07 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


subject: Benjamin Constant, relation: sex or gender, {'2hop_question': 'Who is the spouse of Benjamin Constant?', '2hop_answer': 'Charlotte von Hardenberg', '3hop_question': 'What is the nationality of the spouse of Benjamin Constant?', '3hop_answer': 'German', '4hop_question': 'In which country is the nationality of the spouse of Benjamin Constant recognized?', '4hop_answer': 'Germany', '5hop_question': 'What is the capital city of the country where the nationality of the spouse of Benjamin Constant is recognized?', '5hop_answer': 'Berlin', '6hop_question': 'Which famous landmark is located in the capital city of the country where the nationality of the spouse of Benjamin Constant is recognized?', '6hop_answer': 'Brandenburg Gate'}


...

subject: Martin Karplus, relation: doctoral advisor, {'2hop_question': 'Who was the doctoral advisor of Martin Karplus?', '2hop_answer': 'Linus Pauling', '3hop_question': 'Which university did the doctoral advisor of Martin Karplus work at?', '3hop_answer': 'California Institute of Technology', '4hop_question': 'In which city is the university located where the doctoral advisor of Martin Karplus worked?', '4hop_answer': 'Pasadena', '5hop_question': 'In which state is the city located where the university is situated that the doctoral advisor of Martin Karplus worked at?', '5hop_answer': 'California', '6hop_question': 'In which country is the state located where the city is situated that contains the university where the doctoral advisor of Martin Karplus worked?', '6hop_answer': 'United States'}


09/14/2024 00:03:50 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


subject: Isaac Beeckman, relation: doctoral advisor, {'2hop_question': 'Who was the doctoral student of Isaac Beeckman?', '2hop_answer': 'René Descartes', '3hop_question': 'What is the most famous philosophical work of the doctoral student of Isaac Beeckman?', '3hop_answer': 'Meditations on First Philosophy', '4hop_question': 'In which year was the most famous philosophical work of the doctoral student of Isaac Beeckman published?', '4hop_answer': '1641', '5hop_question': 'Which language was the most famous philosophical work of the doctoral student of Isaac Beeckman originally published in?', '5hop_answer': 'Latin', '6hop_question': 'Which country was the original publication language of the most famous philosophical work of the doctoral student of Isaac Beeckman predominantly used in during the 17th century?', '6hop_answer': 'Roman Catholic Church'}
{'topic': {62: 'human_scientist', 117: 'human_scientist'}, 'subject': {62: 'Buckminster Fuller', 117: 'Buckminster Fuller'}, 'relation':

09/14/2024 00:03:52 - INFO - httpx -   HTTP Request: POST https://n-central-us.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"


subject: Benedikt Buchner, relation: sex or gender, {'2hop_question': 'What is the profession of Benedikt Buchner, who is male?', '2hop_answer': 'Law professor', '3hop_question': 'At which university does the law professor Benedikt Buchner work?', '3hop_answer': 'University of Bremen', '4hop_question': 'In which country is the University of Bremen, where law professor Benedikt Buchner works, located?', '4hop_answer': 'Germany', '5hop_question': 'What is the official language of the country where the University of Bremen, where law professor Benedikt Buchner works, is located?', '5hop_answer': 'German', '6hop_question': 'What is the currency used in the country where the University of Bremen, where law professor Benedikt Buchner works, is located?', '6hop_answer': 'Euro'}
{'topic': {38: 'human_scientist'}, 'subject': {38: 'Semyon Lavochkin'}, 'relation': {38: 'employer'}, 'object': {38: 'NPO Lavochkin'}, 'question': {38: 'What is the employer of Semyon Lavochkin?'}, 'output_meta_llama_3

In [199]:
print(f"Before df_hallu.shape: {df_hallu.shape}")
df_hallu['question_2hop'] = ls_2hop_q
df_hallu['answer_2hop'] = ls_2hop_a
df_hallu['question_3hop'] = ls_3hop_q
df_hallu['answer_3hop'] = ls_3hop_a
df_hallu['question_4hop'] = ls_4hop_q
df_hallu['answer_4hop'] = ls_4hop_a
df_hallu['question_5hop'] = ls_5hop_q
df_hallu['answer_5hop'] = ls_5hop_a
df_hallu['question_6hop'] = ls_6hop_q
df_hallu['answer_6hop'] = ls_6hop_a
print(ls_2hop_a.count('N/A'), ls_3hop_a.count('N/A'), ls_4hop_a.count('N/A'), ls_5hop_a.count('N/A'), ls_6hop_a.count('N/A'))
print(f"After df_hallu.shape: {df_hallu.shape}, saving to {folder_hallu_100}/{domain_topic_name}.csv")
df_hallu.to_csv(f"{folder_hallu_100}/{domain_topic_name}.csv", index=False)

Before df_hallu.shape: (100, 16)
0 0 0 0 0
After df_hallu.shape: (100, 26), saving to ../data/questions/hallucination/mistral_7b_instruct_v0.3_100/human_scientist.csv


In [ ]:
ls_2hop_a.index('N/A'), ls_3hop_a.index('N/A'), ls_4hop_a.index('N/A'), ls_5hop_a.index('N/A'), ls_6hop_a.index('N/A')

In [157]:
# test if there is nan
# for filename in ['technology_software.csv']:
for filename in sorted(os.listdir(folder_hallu_100)):
    if df.isna().sum().sum() > 0:
        print(filename,df.isna().sum().sum())

    # Detect if any part of df_hallu contains 'N/A'
    na_mask = df.applymap(lambda x: x == 'N/A')
    if na_mask.any().any():
        print("'N/A' values found in df_hallu")
        print(na_mask.sum())

## Editing

In [ ]:
df = pd.read_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}.csv")
# df = pd.read_csv(f"../data/questions/wh_only/hallucination_only/mistral_7b_instruct_v0.3.csv")
df.shape

In [13]:
n = 50
targets = df['label'].tolist()[:n]
subjects = df['subject'].tolist()[:n]
questions = df['question'].tolist()[:n]

hparams = ROMEHyperParams.from_hparams('./hparams/ROME/llama3-8b')
# hparams = ROMEHyperParams.from_hparams('./hparams/ROME/gemma-7b')
# hparams = MEMITHyperParams.from_hparams('./hparams/MEMIT/llama3-8b')

hparams.device = 1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=questions,
    # rephrase_prompts=paraphrased_questions,
    target_new=targets,
    subject=subjects,
    # portability_inputs=portability_inputs,
    summary_metrics=True,
    keep_original_weight=True,
    # test_generation=True,
)

json.dump(metrics, open(os.path.join('../results/', f'tmp_ROME_{model_id_format}_results.json'), 'w'), indent=4)
del edited_model
gc.collect()
torch.cuda.empty_cache()

2024-08-12 18:46:29,734 - hallucination_editor - INFO - Instantiating model
08/12/2024 18:46:29 - INFO - hallucination_editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  2%|▏         | 1/50 [00:00<00:33,  1.46it/s]

===== Question: Which tourist attraction was owned by Greece? | Prediction: Knossos. | Label: Parthenon | Evaluation: 0 =====


  4%|▍         | 2/50 [00:01<00:24,  1.92it/s]

...

===== Question: What is the architectural style of Willis Tower? | Prediction: International Style. | Label: International Style | Evaluation: 1 =====
Executing ROME algorithm for the update: [Which tourist attraction's creator is Jan Styka?] -> [ Racławice Panorama]
Computing left vector (u)...
Selected u projection object Jan Styka
Left vector shape: torch.Size([14336])
Computing right vector (v)
Lookup index found: 9 | Sentence: Which tourist attraction's creator is Jan Styka? Racławice Pan | Token: ka
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 2.729 = 2.729 + 0.0 + 0.0 avg prob of [ Racławice Panorama] 0.06627755612134933
loss 2.668 = 2.524 + 0.143 + 0.001 avg prob of [ Racławice Panorama] 0.08082066476345062
loss 2.083 = 2.059 + 0.022 + 0.001 avg prob of [ Racławice Panorama] 0.12921631336212158
loss 1.588 = 1.552 + 0.035 + 0.001 avg prob of [ Racławice Panorama] 0.2130821794271469
loss 3.13 = 3.099 + 0.03 + 0.001 avg prob of [ Racławi

2024-08-12 18:50:12,699 - hallucination_editor - INFO - Execution 49 editing took 2.6805195808410645
08/12/2024 18:50:12 - INFO - hallucination_editor -   Execution 49 editing took 2.6805195808410645


loss 0.045 = 0.033 + 0.011 + 0.001 avg prob of [ Racławice Panorama] 0.9680377840995789
Delta norm: 12.2578125
Change in target norm: 3.064453125 to 12.6328125 => 9.5703125
Division Factor: 3.83984375
Right vector norm: 3.19140625
Right vector shape: torch.Size([4096])
Deltas successfully computed for ['model.layers.5.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.5.mlp.down_proj.weight']


2024-08-12 18:50:13,151 - hallucination_editor - INFO - Evaluation took 0.45148134231567383
08/12/2024 18:50:13 - INFO - hallucination_editor -   Evaluation took 0.45148134231567383
2024-08-12 18:50:13,152 - hallucination_editor - INFO - 49 editing: Which tourist attraction's creator is Jan Styka? -> Racławice Panorama  
 {'pre': {'edit_acc': [0], 'edit_output': ['The National Shrine of the Immaculate Conception.'], 'locality': {}, 'portability': {}, 'yes_no': {}}, 'case_id': 49, 'requested_edit': {'prompt': "Which tourist attraction's creator is Jan Styka?", 'target_new': 'Racławice Panorama', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'yes_no': {}, 'harm_original_text': {}, 'subject': 'Jan Styka'}, 'time': 2.6805195808410645, 'post': {'edit_acc': [1], 'edit_output': ['Racławice Panorama.'], 'locality': {}, 'portability': {}, 'yes_no': {}}}
08/12/2024 18:50:13 - INFO - hallucination_editor -   49 editing: Which tourist attraction's creator is Jan Styka? -> Rac

===== Question: Which tourist attraction's creator is Jan Styka? | Prediction: Racławice Panorama. | Label: Racławice Panorama | Evaluation: 1 =====
Metrics Summary:  {'pre': {'edit_acc': 0.0}, 'post': {'edit_acc': 0.86}, 'time': 3.556901044845581}
